In [1]:
# Acá van las librerías que vamos a ocupar :)
import numpy as np                           # Útil para procesos matemáticos
from sympy import *                          # Útil para calcular derivadas  
import matplotlib.pyplot as plt              # Plotting 
from datetime import date as dt              # Para trabajar con fechas y diferencias de fechas 
from datetime import datetime
from dateutil import relativedelta as rd

## Proyecto Tarea 4

### Parte I

El objetivo de esta parte del proyecto es producir un código capaz de determinar la tasa interna de retorno (TIR), la duración y la convexividad de un bono con cupones, con una precisión de al menos 3 cifras significativas. 

El valor de un bono con cupones viene dado por la siguiente expresión, vista en clases:

$$ V(t)=Pe^{-y(T-t)}+\sum_{j=1}^N C_j e^{-y(t_j-t)}, $$

donde $P$ es el monto principal, $T$ es el tiempo final, $t$ es el tiempo actual, C_j es lo pagado por cada cupon, N es el numero de cupones y $t_j$ los tiempos en los que debemos pagar cada cupon e $y$ es el TIR. Podemos expresar $t_j-t=j/A$, dado intervalos de tiempos equitativos. Adémas, diremos que todos los $C_j$ son iguales y estan dados por:

$$ C_j=C=\frac{P r}{A},$$

donde $r$ es la taza anual de los cupones y $A$ es 1 o 2 dependiendo de si los cupones son anuales o bianuales, respectivamente. La Duración y la Convexidad vienen dadas por:

$$ D=-\frac{1}{V}\frac{dV}{dy}, \qquad C=\frac{1}{2V}\frac{d^2V}{dy^2}. $$

Calculando, obtenemos:

$$ D=\frac{P}{V}(T-t)e^{-y(T-t)} + \sum_{j=1}^N\frac{C_j}{V}(t_j-t)e^{-y(t_j-t)} , $$

$$ C=\frac{P}{2V}(T-t)^2e^{-y(T-t)} + \sum_{j=1}^N\frac{C_j}{2V}(t_j-t)^2e^{-y(t_j-t)} . $$

Partamos por los inputs. Para la información a recibir para el cálculo no voy a hacer una interfaz esta vez, pero dejo abajo una celda comentada donde se pueden introducir los valores. Razón: más rapido para ambos :P 

In [8]:
# Partamos definiendo un par de funciones útiles 

# Función que convierte las fechas a objetos datetime 
def str_to_dt(date): 
    date = date.split('-')
    return dt(int(date[0]), int(date[1]), int(date[2]))

# Función para obtener días entre dos fechas
def delta_days(d1,d2):
    diff = d1 - d2
    return diff.days
    
# Función que retorna factor de división según tipo de bono
def div_factor(tipo_de_bono):
    if tipo_de_bono =='a': return 1
    elif tipo_de_bono == 'b': return 2
    return None

# Función porcentage a número (A prueba de ingenieros comerciales)
def percent_to_num(percent): return 100*percent # xD

# Espacio para ingresar variables 
# Algunas van dentro de funciones que me entregan un valor trabajable
p = 100000000                  # Monto Principal 
t0 = str_to_dt('2023-6-1')     # Fecha actual (formato: año-mes-día)
#t0 = datetime.datetime.now()  # También puedes poner automaticamente hoy descomentando esta linea
tf = str_to_dt('2024-6-1')     # Fecha de madurez  (formato: año-mes-día)
a_or_b = div_factor('a')       # Cupones anuales (Ingresar 'a') o bi anuales (Ingresar 'b') 
r = percent_to_num(3)          # Tasa anual de los cupones (en %)
v0 = 96000000                  # precio actual del bono


In [ ]:
# Calculemos lo que nos piden

# Primero definimos la función para el valor del bono
def value_b(y, p, t0, tf, a_or_b, r, v0):
    dt = int(delta_days(tf,t0)/365)
    n_cupones = dt*a_or_b
    [(p*r/a_or_b)]
    return p*np.e^(-y*dt)+np.sum()

def tir(p, t0, tf, a_or_b, r, v0):
    ''' La idea de esta función es setear n valores para el TIR entre 1 y 0,
        calcular la diferencia del valor del cupon para esos TIRs respecto
        al valor deseado, tomar el con menos error y volver a iterar bajo valores
        cecanos a este. 
        '''
    y_start = 0.5      # definimos un valor inicial para el TIR
